In [56]:
import os
import yaml
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# Show all rows from pandas
# pd.set_option('display.max_rows', None)
import xlwings as xw

# PARAMETRES ET CONSTANTES

In [57]:
with open('./input_data/infos_générales.yaml', 'r') as file:
        data = yaml.safe_load(file)

In [58]:
data

{'main_infos': {'titre_projet': 'Complexe sportif Diderot',
  'soustitre_projet': 'Lot 4 Ossature et Veture bois',
  'redacteur': 'L. Toussaint',
  'email_redacteur': 'ltoussaint@simonin.com',
  'verificateur': 'JB. Journot',
  'email_verificateur': 'jbjournot@simonin.com',
  'numero_affaire': '2404-234',
  'note_hypotheses': '2409-500',
  'adresse_projet': 'Gymnase Diderot\nRue de Cologne\n25000 BESANCON\n',
  'numero_reference': 'GBM Diderot',
  'numero_doc': '2409-501',
  'maitrise_ouvrage': 'Ville de Besançon\nLa City - 4 rue Gabriel Plançon\nBESANCON CEDEX 25043\n',
  'architectes': 'LIEUX FAUVES / Architecture et paysage\n43, rue des Hérideaux\nLYON 69008\n04 72 78 80 80\nc.lacoste@lieuxfauves.com\n',
  'bureau_etudes_controle': 'SYNAPSE / Structure, Fluides, VRD, CSSI\n200, Bd de la résistance 71000 MACON\nvbourillon@synapse-construction.com\n03 85 38 66 22\n'},
 'tableau_charges': {'axe_RFx': {'valeur_mini_affichage_kN': 1},
  'axe_RFy': {'valeur_mini_affichage_kN': 1},
  'axe_

In [59]:
# # Avec MISE A JOUR DU LATEX
# # Create the main_infos.tex file
# with open('latex_files/main_infos.tex', 'w') as file:
#     file.write(f"""
# \\maitriseouvrage{{{data['main_infos']['maitrise_ouvrage']}}}
# \\architectes{{{data['main_infos']['architectes']}}}
# \\bureauetudescontrole{{{data['main_infos']['bureau_etudes_controle']}}}
# \\titreprojet{{{data['main_infos']['titre_projet']}}}
# \\soustitreprojet{{{data['main_infos']['soustitre_projet']}}}
# \\redacteur{{{data['main_infos']['redacteur']}}}
# \\emailredacteur{{{data['main_infos']['email_redacteur']}}}
# \\verificateur{{{data['main_infos']['verificateur']}}}
# \\emailverificateur{{{data['main_infos']['email_verificateur']}}}
# \\numeroaffaire{{{data['main_infos']['numero_affaire']}}}
# \\notehypotheses{{{data['main_infos']['note_hypotheses']}}}
# \\adresseprojet{{{data['main_infos']['adresse_projet']}}}
# \\numeroreference{{{data['main_infos']['numero_reference']}}}
# \\numerodoc{{{data['main_infos']['numero_doc']}}}
# """)

In [60]:
# Create the work_scope.tex file


In [104]:
def split_premier(x, sep):
    try:
        premier = x.split(sep)[0]
    except:
        premier = np.nan
    return premier

def split_deuxieme(x, sep):
    try:
        deuxieme = x.split(sep)[1]
    except:
        deuxieme = np.nan
    return deuxieme

def create_latex_table(df:pd.DataFrame, numero_cas:int):
        # df = df_ddc[df_ddc['Type_chargement']==choix_chargement] 
        df['RFx_kN'] = df.apply(lambda x: '' if abs(x['RFx_kN'])<data['tableau_charges']['axe_RFx']['valeur_mini_affichage_kN'] else -round(x['RFx_kN'],1), axis=1)
        df['RFy_kN'] = df.apply(lambda x: '' if abs(x['RFy_kN'])<data['tableau_charges']['axe_RFy']['valeur_mini_affichage_kN'] else -round(x['RFy_kN'],1), axis=1)
        df['RFz_kN'] = df.apply(lambda x: '' if abs(x['RFz_kN'])<data['tableau_charges']['axe_RFz']['valeur_mini_affichage_kN'] else -round(x['RFz_kN'],1), axis=1)
        # Replace in column 'Nom point' all '_' by '\_'
        df['Nom point'] = df['Nom point'].apply(lambda x: x.replace('_', '\\_'))
        # Replace all columns names containing '_' by '\\_' in df
        df.columns = df.columns.str.replace('_', '\\_')
        df[['Nom point','X', 'Y', 'Z', 'RFx\\_kN', 'RFy\\_kN', 'RFz\\_kN']].to_latex(f'latex_files/table_{numero_cas}.tex', 
                                                                                    index=False, header=True, bold_rows=True,
                                                                                    float_format="%.1f")

def create_3D_plot(df, numero_cas:int):
        df['RFx_kN'] = df.apply(lambda x: '' if abs(x['RFx_kN'])<data['tableau_charges']['axe_RFx']['valeur_mini_affichage_kN'] else -round(x['RFx_kN'],1), axis=1)
        df['RFy_kN'] = df.apply(lambda x: '' if abs(x['RFy_kN'])<data['tableau_charges']['axe_RFy']['valeur_mini_affichage_kN'] else -round(x['RFy_kN'],1), axis=1)
        df['RFz_kN'] = df.apply(lambda x: '' if abs(x['RFz_kN'])<data['tableau_charges']['axe_RFz']['valeur_mini_affichage_kN'] else -round(x['RFz_kN'],1), axis=1)
        for reaction in ['RFx_kN', 'RFy_kN', 'RFz_kN']:
                ###############################################
                # Create figure and 3D axis
                fig = plt.figure()
                ax = fig.add_subplot(111, projection='3d')
                # Plot the 3D data
                ax.scatter(xs=df.X, ys=df.Y, zs=df.Z, 
                        marker='o', edgecolor='black',  facecolor='none', s=10, alpha=1)      
                ax.set_yticklabels([])
                ax.set_xticklabels([])
                ax.set_zticklabels([])
                ax.set_xticks([])
                ax.set_yticks([])
                ax.set_zticks([])
                ax.set_xlabel('X axis')
                ax.set_ylabel('Y axis')
                ax.set_zlabel('Z axis')
                # Change the angle of vision (elevation, azimuth)
                ax.view_init(elev=data['graph_3D']['elevation'], azim=data['graph_3D']['azimuth'])#) 
                # Fill cells in column Rfz_kN with np.nan if value is ""
                # GR1.df_ddc_individuelle['RFz_kN'] = GR1.df_ddc_individuelle['RFz_kN'].apply(lambda x: np.nan if x=='' else x)
                if reaction == 'RFz_kN':
                        for i, txt in df.iterrows():
                                if txt['RFz_kN'] != '':        
                                        ax.plot(xs= [txt['X'], txt['X']], 
                                                ys=[txt['Y'], txt['Y']], 
                                                zs=[txt['Z'], txt['Z']+txt['RFz_kN']*data['graph_3D']['scale_factor_Z']], 
                                                marker=None, color='r', linestyle='-', linewidth=1)  
                                        ax.text(txt['X'], txt['Y'], txt['Z']+txt['RFz_kN']*data['graph_3D']['scale_factor_Z'], 
                                                txt['RFz_kN'], ha='center', va='top', size=5, color='r')
                        fig.savefig(f"assets/img/graph3D_charges_cas_{numero_cas}_RFz_kN.png", dpi=300)
                        plt.show()
                elif reaction == 'RFy_kN':
                        for i, txt in df.iterrows():
                                if txt['RFy_kN'] != '':        
                                        ax.plot(xs= [txt['X'], txt['X']], 
                                                ys=[txt['Y'], txt['Y']+txt['RFy_kN']*data['graph_3D']['scale_factor_Y']], 
                                                zs=[txt['Z'], txt['Z']], 
                                                marker=None, color='g', linestyle='-', linewidth=1)  
                                        ax.text(txt['X'], txt['Y']+txt['RFy_kN']*data['graph_3D']['scale_factor_Y'], txt['Z'], 
                                                txt['RFy_kN'], ha='center', va='bottom', size=5, color='g')
                        fig.savefig(f"assets/img/graph3D_charges_cas_{numero_cas}_RFy_kN.png", dpi=300)
                        plt.show()
                elif reaction == 'RFx_kN':
                        for i, txt in df.iterrows():
                                if txt['RFx_kN'] != '':        
                                        ax.plot(xs= [txt['X'], txt['X']+txt['RFx_kN']*data['graph_3D']['scale_factor_X']], 
                                                ys=[txt['Y'], txt['Y']], 
                                                zs=[txt['Z'], txt['Z']], 
                                                marker=None, color='b', linestyle='-', linewidth=1)  
                                        ax.text(txt['X']+txt['RFx_kN']*data['graph_3D']['scale_factor_X'], txt['Y'], txt['Z'], 
                                                txt['RFx_kN'], ha='left', va='center', size=5, color='b')
                        fig.savefig(f"assets/img/graph3D_charges_cas_{numero_cas}_RFx_kN.png", dpi=300)
                        plt.show()
        
def create_latex_cas(numero_cas:int):
    nom = data['cas'][numero_cas]['nom']
    table  = f'table_{numero_cas}.tex'
    with open(f'latex_files/cas_{numero_cas}.tex', 'w') as file:
        file.write(f"""
\\section{{{nom}}}
\\import{{latex_files/}}{{{table}}}

\\begin{{figure}}[H] % Pour insérer une figure
     \\centering % Pour centrer l'image
     \\includegraphics[width=0.7\\textwidth]{{assets/img/graph3D_charges_cas_{{{numero_cas}}}_RFx_kN.png}} % Pour insérer l'image
     \\caption{{{nom}\_RFx\_kN}} % Légende de l'image
\\end{{figure}}

\\begin{{figure}}[H] % Pour insérer une figure
     \\centering % Pour centrer l'image
     \\includegraphics[width=0.7\\textwidth]{{assets/img/graph3D_charges_cas_{{{numero_cas}}}_RFy_kN.png}} % Pour insérer l'image
     \\caption{{{nom}\_RFy\_kN}} % Légende de l'image
\\end{{figure}}

\\begin{{figure}}[H] % Pour insérer une figure
     \\centering % Pour centrer l'image
     \\includegraphics[width=0.7\\textwidth]{{assets/img/graph3D_charges_cas_{{{numero_cas}}}_RFz_kN.png}} % Pour insérer l'image
     \\caption{{{nom}\_RFz\_kN}} % Légende de l'image
\\end{{figure}}

""")
                   


#########################################################################

class Group():
    def __init__(self, Groupe_id):
        self.groupe_id = Groupe_id
    
    def load_coordonnees(self):
        df = pd.read_csv(f'./input_data/{self.groupe_id}_Coordonées Points.csv', sep=';', header=2)  
        # Remove first line which contains the unit
        df = df.drop(0)
        df_coord_0 = df[['Nom point', 'X', 'Y', 'Z']]
        df_coord_1 = df[['Nom point.1', 'X.1', 'Y.1', 'Z.1']]
        # Rename columns of df_coord_1
        df_coord_1.columns = ['Nom point', 'X', 'Y', 'Z']
        df_coord = pd.concat([df_coord_0, df_coord_1], axis=0)
        # convert all values from df_coord to integer
        df_coord['X'] = df_coord['X'].apply(lambda x: int(float(x)))
        df_coord['Y'] = df_coord['Y'].apply(lambda x: int(float(x)))
        df_coord['Z'] = df_coord['Z'].apply(lambda x: int(float(x)))
        
        # Reset index
        df_coord.reset_index(drop=True, inplace=True)
        # From df_coord, Split each row of column 'Nom point' to get axe_lettre and axe_chiffre
        df_coord['axe_lettre'] = df_coord.apply(lambda x: split_premier(x['Nom point'], '_'), axis=1)
        df_coord['axe_chiffre'] = df_coord.apply(lambda x: split_deuxieme(x['Nom point'], '_'), axis=1) 

        # Remove from df all points where 'Num point' is -
        df_coord = df_coord[df_coord['Nom point'] != '-']

        self.df_coordonnees = df_coord


    def load_ddc_individuelle(self):
        df = pd.read_csv(f'./input_data/{self.groupe_id}_DDC Individuelle.csv', sep=';', header=2)
        # Remove first line which contains the unit
        df = df.drop(0)
        # Remove row if column Point contains "Total
        df = df[df['Point'] != 'Total']
        df['RFx'] = df['RFx'].apply(lambda x: int(float(x)))
        df['RFy'] = df['RFy'].apply(lambda x: int(float(x)))
        df['RFz'] = df['RFz'].apply(lambda x: int(float(x)))
        df['RFx_kN'] = df['RFx']/100
        df['RFy_kN'] = df['RFy']/100
        df['RFz_kN'] = df['RFz']/100
        # Replace all values in column RFx_kN by "" if absolute value of RFz_kN is less than data['tableau_charges']['axe_RFx']['valeur_mini_affichage_kN']
        df['Iteration_#'] = df.apply(lambda x: int(split_premier(x['Cas'],'-')), axis=1)
        df['Type_chargement'] = df.apply(lambda x: split_deuxieme(x['Cas'], '-'), axis=1)
        # Reset index
        df.reset_index(drop=True, inplace=True)
        
        self.df_ddc_individuelle = df
    
    def plot_graphes_reperage_points(self):
        # Graph 2D
        
        # Create a 2d plot of the points X and Y
        fig, ax = plt.subplots()
        ax.scatter(self.df_coordonnees['X'], self.df_coordonnees['Y'])
        for i, txt in self.df_coordonnees.iterrows():
            ax.text(txt['X'], txt['Y'], txt['Nom point'], ha='left', va='bottom')
        # Hide the axes
        ax.set_axis_off()
        fig.savefig('assets/img/reperage_points_appuis.png')
        plt.show()

        # Graph 3D
        # Plot 3D plot of the df_ddc_individuelle 
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(self.df_coordonnees['X'], self.df_coordonnees['Y'], self.df_coordonnees['Z'], c='r', marker='o')
        #
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_yticklabels([])
        ax.set_xticklabels([])
        ax.set_zticklabels([])
        # ax.set_xticks([])
        # ax.set_yticks([])
        # ax.set_zticks([])

        for i, txt in self.df_coordonnees.iterrows():
            ax.text(txt['X'], txt['Y'],txt['Z'], txt['Nom point'], ha='left', va='bottom')
        #
        ax.view_init(elev=data['graph_3D']['elevation'], azim=data['graph_3D']['azimuth']) 
        #
        fig.savefig('assets/img/reperage_points_appuis_3D.png')
        plt.show()
    
    def merge_points_charges_individuelles(self):
        # df= df.sort_values(by=['axe_lettre', 'axe_chiffre'])
        self.df_ddc_individuelle_points = pd.merge(self.df_coordonnees, self.df_ddc_individuelle, on='Nom point', how='left')
        self.df_ddc_individuelle_points['axe_chiffre'] = self.df_ddc_individuelle_points['axe_chiffre'].astype(int)
        self.df_ddc_individuelle_points = self.df_ddc_individuelle_points.sort_values(by=['axe_lettre', 'axe_chiffre'])

    def sort_df(self, numero_cas:int)-> pd.DataFrame:
        cas_choisi = data['cas'][numero_cas]
        print('Le Cas choisi est :', cas_choisi)
        # composantes = ['1-G']
        # type_combinaison = 'sum' # sum ou max ou max_absolu
        df = self.df_ddc_individuelle_points[self.df_ddc_individuelle_points['Cas'].isin(cas_choisi['composantes'])]
        df = df[['Nom point', 'RFx_kN', 'RFy_kN', 'RFz_kN']]
        # For each point "Nom point", get the sum of the values of the column 'RFx_kN' and 'RFy_kN' and 'RFz_kN'
        df_cas = df.pivot_table(index='Nom point', values=['RFx_kN', 'RFy_kN', 'RFz_kN'], aggfunc=cas_choisi['type_combinaison'])
        # Move index to a column
        df_cas.reset_index(inplace=True)
        # Merge df_cas with df_coordonnees
        df_cas = pd.merge(df_cas,self.df_coordonnees, on='Nom point', how='left')
        df_cas['axe_chiffre'] = df_cas['axe_chiffre'].astype(int)
        df_cas = df_cas.sort_values(by=['axe_lettre', 'axe_chiffre'])
        df_cas.reset_index(drop=True, inplace=True)
        return df_cas
    
    
            


https://matplotlib.org/stable/api/_as_gen/mpl_toolkits.mplot3d.axes3d.Axes3D.stem.html#mpl_toolkits.mplot3d.axes3d.Axes3D.stem

# Liste des fichiers csv dans input_data

In [105]:
liste_csv = glob.glob('./input_data/*.csv')
liste_csv

['./input_data\\Groupe 1_Coordonées Points.csv',
 './input_data\\Groupe 1_DDC individuelle EX & EY + DDC combinée ELU STR & Séisme.csv',
 './input_data\\Groupe 1_DDC individuelle.csv',
 './input_data\\Groupe 2_Coordonées Points.csv',
 './input_data\\Groupe 2_DDC individuelle EX & EY + DDC combinée ELU STR & Séisme.csv',
 './input_data\\Groupe 2_DDC individuelle.csv']

## Nombre de groupe

In [108]:
groupe = list()
for f in liste_csv:
    gr_id = f.split('\\')[-1].split('_')[0]
    if gr_id not in groupe:
        groupe.append(gr_id)
groupe

['Groupe 1', 'Groupe 2']

In [109]:
GR1 = Group(Groupe_id=groupe[0])
GR1.load_coordonnees()
GR1.load_ddc_individuelle()

## Coordonnées des Points

In [110]:
GR1.df_coordonnees

,Nom point,X,Y,Z,axe_lettre,axe_chiffre
94,B_1,0,3160,11112,B,1
95,D_1,0,12760,11112,D,1
96,F_1,0,22360,11112,F,1
97,H_1,0,31960,11112,H,1
98,J_1,0,41560,11112,J,1
99,A_1,0,0,5790,A,1
100,A_2,3250,0,5790,A,2
101,A_3,6500,0,5790,A,3
102,K_4,9750,44720,0,K,4
103,K_5,13000,44720,0,K,5


In [111]:
# Avec MISE A JOUR DU LATEX
# GR1.plot_graphes_reperage_points()

In [112]:
# Order
GR1.df_coordonnees = GR1.df_coordonnees.sort_values(by=['axe_lettre', 'axe_chiffre'])
GR1.df_coordonnees['Nom point'].unique()

array(['A_1', 'A_10', 'A_11', 'A_2', 'A_3', 'A_4', 'A_5', 'A_6', 'A_7',
       'A_8', 'A_9', 'B_1', 'B_11', 'C_1', 'C_11', 'D_1', 'D_11', 'E_1',
       'E_11', 'F_1', 'F_11', 'G_1', 'G_11', 'H_1', 'H_11', 'I_1', 'I_11',
       'J_1', 'J_11', 'K_1', 'K_10', 'K_11', 'K_2', 'K_3', 'K_4', 'K_5',
       'K_6', 'K_7', 'K_8', 'K_9'], dtype=object)

## Descente de charge

In [113]:
GR1.merge_points_charges_individuelles()
GR1.df_ddc_individuelle_points[:20]

,Nom point,X,Y,Z,axe_lettre,axe_chiffre,Point,Cas,RFx,RFy,RFz,RFx_kN,RFy_kN,RFz_kN,Iteration_#,Type_chargement
0,A_1,0,0,5790,A,1,309,1-G,7,3,515,0.07,0.03,5.15,1,G
1,A_1,0,0,5790,A,1,309,2-Q,0,0,0,0.00,0.00,0.00,2,Q
2,A_1,0,0,5790,A,1,309,3-S,3,1,205,0.03,0.01,2.05,3,S
3,A_1,0,0,5790,A,1,309,13-W,-3,5,51,-0.03,0.05,0.51,13,W
4,A_1,0,0,5790,A,1,309,15-W3,0,27,-148,0.00,0.27,-1.48,15,W3
5,A_1,0,0,5790,A,1,309,17-W5,-1,1,50,-0.01,0.01,0.50,17,W5
6,A_1,0,0,5790,A,1,309,19-W7,4,-30,-148,0.04,-0.30,-1.48,19,W7
21,A_2,3250,0,5790,A,2,308,1-G,6,3,671,0.06,0.03,6.71,1,G
22,A_2,3250,0,5790,A,2,308,2-Q,0,0,0,0.00,0.00,0.00,2,Q
23,A_2,3250,0,5790,A,2,308,3-S,2,2,339,0.02,0.02,3.39,3,S


In [114]:
GR1.df_ddc_individuelle_points.Cas.unique()

array(['1-G', '2-Q', '3-S', '13-W', '15-W3', '17-W5', '19-W7'],
      dtype=object)

In [115]:
df = GR1.sort_df(numero_cas=1)
df

Le Cas choisi est : {'nom': 'Charges permanentes G', 'type_combinaison': 'sum', 'composantes': ['1-G']}


,Nom point,RFx_kN,RFy_kN,RFz_kN,X,Y,Z,axe_lettre,axe_chiffre
0,A_1,0.07,0.03,5.15,0,0,5790,A,1
1,A_2,0.06,0.03,6.71,3250,0,5790,A,2
2,A_3,-1.02,0.10,13.15,6500,0,5790,A,3
3,A_4,3.14,0.04,14.07,9750,0,0,A,4
4,A_5,0.02,0.02,24.64,13000,0,0,A,5
5,A_6,-3.30,0.24,48.06,16250,0,0,A,6
6,A_7,0.01,0.00,54.84,19500,0,0,A,7
7,A_8,4.98,0.04,29.64,22750,0,0,A,8
8,A_9,-0.01,-0.04,24.02,26000,0,0,A,9
9,A_10,-1.30,0.07,33.44,29250,0,0,A,10


In [116]:
# create_latex_table(df = GR1.sort_df(numero_cas=1), 
#                     numero_cas=1)

In [117]:
create_3D_plot(df=GR1.sort_df(numero_cas=1), 
                    numero_cas=1)

Le Cas choisi est : {'nom': 'Charges permanentes G', 'type_combinaison': 'sum', 'composantes': ['1-G']}


C:\Users\nmorand\AppData\Local\Temp\ipykernel_19472\838061733.py:85: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\nmorand\AppData\Local\Temp\ipykernel_19472\838061733.py:74: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\nmorand\AppData\Local\Temp\ipykernel_19472\838061733.py:63: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [120]:
# Open the existing workbook
workbook = xw.Book('TEMPLATE_DDC.xlsx')
# workbook = xw.Book()

In [121]:
config_sheet = workbook.sheets['config']

In [122]:
last_row = config_sheet.range('A1').end('down').row
last_col = config_sheet.range('A1').end('right').column
config = {}
for row in range(2, last_row+1):
    config[config_sheet.range((row,1)).value] = config_sheet.range((row,2)).value
print(last_row, last_col)

8 16384


In [123]:
config

{'cas_1_nom': 'Charges permanentes G',
 'cas_1_reperage_points_3D': 'HZ39',
 'cas_1_RFx_kN': 'JA39',
 'cas_1_RFy_kN': 'HZ62',
 'cas_1_RFz_kN': 'JA62',
 'cas_1_premier_point_ligne': 11.0,
 'cas_1_premier_point_colonne': 235.0}

In [124]:
# Select the 'DDC' sheet
sheet = workbook.sheets['DdC(FRA)']
# sheet = workbook.sheets.active

In [125]:
sheet.pictures

Pictures([<Picture 'Picture 1196' in <Sheet [TEMPLATE_DDC.xlsx]DdC(FRA)>>, <Picture 'Picture 3' in <Sheet [TEMPLATE_DDC.xlsx]DdC(FRA)>>, <Picture 'Picture 4' in <Sheet [TEMPLATE_DDC.xlsx]DdC(FRA)>>, ...])

In [126]:
df = GR1.sort_df(numero_cas=1)
df

Le Cas choisi est : {'nom': 'Charges permanentes G', 'type_combinaison': 'sum', 'composantes': ['1-G']}


,Nom point,RFx_kN,RFy_kN,RFz_kN,X,Y,Z,axe_lettre,axe_chiffre
0,A_1,0.07,0.03,5.15,0,0,5790,A,1
1,A_2,0.06,0.03,6.71,3250,0,5790,A,2
2,A_3,-1.02,0.10,13.15,6500,0,5790,A,3
3,A_4,3.14,0.04,14.07,9750,0,0,A,4
4,A_5,0.02,0.02,24.64,13000,0,0,A,5
5,A_6,-3.30,0.24,48.06,16250,0,0,A,6
6,A_7,0.01,0.00,54.84,19500,0,0,A,7
7,A_8,4.98,0.04,29.64,22750,0,0,A,8
8,A_9,-0.01,-0.04,24.02,26000,0,0,A,9
9,A_10,-1.30,0.07,33.44,29250,0,0,A,10


In [130]:
# Chargement des données dans le tableau
df = GR1.sort_df(numero_cas=1)
for i, txt in df.iterrows():
    sheet.range((11+i,235)).value = txt['Nom point'] 
    sheet.range((11+i,236)).value = txt['X']
    sheet.range((11+i,237)).value = txt['Y']
    sheet.range((11+i,238)).value = txt['Z']
    sheet.range((11+i,240)).value = txt['RFx_kN']
    sheet.range((11+i,242)).value = txt['RFy_kN']
    sheet.range((11+i,244)).value = txt['RFz_kN']
    if i>=24:
        break

Le Cas choisi est : {'nom': 'Charges permanentes G', 'type_combinaison': 'sum', 'composantes': ['1-G']}


In [129]:
numero_cas = 1
# Chargement du titre du cas 1
sheet.range('HY6').value = f'{numero_cas+1}.DESCENTE DE CHARGES: {config[f"cas_{str(numero_cas)}_nom"]}'

# Chargement des données dans le tableau


# Chargement des images dans le cas 1
image_path = r'C:\Users\nmorand\OneDrive - SIMONIN SAS\_Descente_de_charge\assets\img\reperage_points_appuis_3D.png'
sheet.pictures.add(image_path, left=sheet.range(config['cas_1_reperage_points_3D']).left, top=sheet.range(config['cas_1_reperage_points_3D']).top)

for graphe in ['cas_1_RFx_kN', 'cas_1_RFy_kN', 'cas_1_RFz_kN']:
    # Define the path to the image
    image_path = r'C:\Users\nmorand\OneDrive - SIMONIN SAS\_Descente_de_charge\assets\img\graph3D_charges_'+graphe+'.png'
    print(type(image_path))
    # image_path = os.path.join(directory, file_name)
    # image_path = directory + "\\" + file_name
    print(image_path)
    # Insert the image at a specific cell (e.g., B2)
    sheet.pictures.add(image_path, left=sheet.range(config[graphe]).left, top=sheet.range(config[graphe]).top)

# Save the workbook (optional: save under a new name to avoid overwriting)
# workbook.save('TEMPLATE_DDC_with_image.xls')

# Close the workbook
# workbook.close()


<class 'str'>
C:\Users\nmorand\OneDrive - SIMONIN SAS\_Descente_de_charge\assets\img\graph3D_charges_cas_1_RFx_kN.png
<class 'str'>
C:\Users\nmorand\OneDrive - SIMONIN SAS\_Descente_de_charge\assets\img\graph3D_charges_cas_1_RFy_kN.png
<class 'str'>
C:\Users\nmorand\OneDrive - SIMONIN SAS\_Descente_de_charge\assets\img\graph3D_charges_cas_1_RFz_kN.png


## Creation du Cas .tex

In [114]:
create_latex_cas(numero_cas=2)

In [ ]:
GR1.df_ddc_individuelle

In [ ]:
GR1.df_ddc_individuelle

In [ ]:
GR1.df_ddc_individuelle.describe()